# Asset News Sentiment Analyzer

In [25]:
import os
from GoogleNews import GoogleNews
from textblob import TextBlob
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import time
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from openbb import obb

In [26]:
pd.options.plotting.backend="plotly"

In [27]:
obb.user.preferences.output_type = "dataframe"

In [28]:
def get_stock_data(symbol, start_date=None, end_date=None):
    data = obb.equity.price.historical(
        symbol,
        start_date=start_date,
        end_date=end_date,
        provider="yfinance",
    )
    data.reset_index(inplace=True)
    return data

In [51]:
def get_stock_data_1m(symbol, start_date=None, end_date=None):
    data = obb.equity.price.historical(
        symbol,
        start_date=start_date,
        end_date=end_date,
        interval="1m",
        provider="yfinance",
    )
    data.reset_index(inplace=True)
    return data

In [52]:
x = get_stock_data_1m("SPY", "2024-11-25","2024-11-26")

In [53]:
x

,date,open,high,low,close,volume
0,2024-11-25 09:30:00,599.520020,599.640015,598.950012,599.000000,2414375
1,2024-11-25 09:31:00,599.000000,599.136597,598.799988,599.099976,375076
2,2024-11-25 09:32:00,599.140015,599.760010,599.028320,599.690002,274140
3,2024-11-25 09:33:00,599.664978,599.940002,599.650024,599.900024,189964
4,2024-11-25 09:34:00,599.890015,600.150024,599.880005,600.070007,332947
...,...,...,...,...,...,...
534,2024-11-26 11:55:00,599.820007,599.869995,599.741028,599.789978,51001
535,2024-11-26 11:56:00,599.705017,599.799988,599.660278,599.705017,23865
536,2024-11-26 11:57:00,599.710022,599.749878,599.530029,599.670105,89661
537,2024-11-26 11:58:00,599.640015,599.719971,599.619995,599.719971,18231


In [29]:
def get_data(name):
    
    file_path = os.path.join("name.csv")
    
    return pd.read_csv(
        file_path,
        index_col="date"
    )

In [30]:
def sentiment_actif(actif, date_mesure):
  #date in format "mm/dd/yyyy"
  time.sleep(30)
  
  googlenews=GoogleNews()
  googlenews.set_time_range(date_mesure,date_mesure)
  googlenews.search(actif)
  news=googlenews.results()
  
  #calculo
  sentiment_score=[]
  for elt in news:
    text=TextBlob(elt['title'])
    sentiment_score.append(text.sentiment.polarity)
  
  neg_value=len(list(filter(lambda x: (x < 0), sentiment_score)))
  pos_value=len(list(filter(lambda x: (x >= 0), sentiment_score)))

  if pos_value>neg_value:
    pos_neg=1
  else:
    if pos_value==neg_value:
      pos_neg=0
    else:
      pos_neg=-1
  
  return (sum(sentiment_score)/len(sentiment_score),pos_neg)

In [31]:
def conversion_date(date_c):
  #from "YYY-MM-DD" to "MM-DD-YYY"
  return date_c.strftime("%m/%d/%Y")

In [32]:
def prix_actif_sentiment(actif, ticker, date_debut, date_fin):
  
  df= get_stock_data(ticker, date_debut, date_fin)
  
  df["Date_mesure"]=df.apply(lambda x: conversion_date(x["date"]),axis=1)
  
  df["sentiment_score_avg"]=df.apply(lambda x: sentiment_actif(actif, x["Date_mesure"])[0],axis=1)

  df["sentiment_score_maj"]=df.apply(lambda x: sentiment_actif(actif, x["Date_mesure"])[1],axis=1)
  
  file_path = os.path.join("btc.csv")
  df.to_csv(file_path, index=True)
  
  return df.set_index("date")

In [56]:
mydf=prix_actif_sentiment("celsius","CELH","2024-11-25","2024-11-29")

In [57]:
mydf

,open,high,low,close,volume,Date_mesure,sentiment_score_avg,sentiment_score_maj
date,,,,,,,,
2024-11-25,30.340000,31.110001,29.700001,29.750000,14988900,11/25/2024,-0.072,1
2024-11-26,29.389999,29.440001,27.370001,27.600000,10645100,11/26/2024,-0.072,1
2024-11-27,27.690001,28.450001,27.450001,28.370001,5339500,11/27/2024,-0.072,1


In [38]:
row=mydf.to_csv()

with open('sentiment_prix.csv','w') as fd:
    fd.write(row)

In [39]:
mydf2 = pd.read_csv('sentiment_prix.csv')

In [43]:
mydf2 = mydf2.set_index("date")

In [44]:
mydf2

,open,high,low,close,volume,Date_mesure,sentiment_score_avg,sentiment_score_maj
date,,,,,,,,
2022-10-01,19431.105469,19471.154297,19231.082031,19312.095703,18719537670,10/01/2022,-0.003864,1
2022-10-02,19311.849609,19370.308594,18970.621094,19044.107422,20765955327,10/02/2022,-0.003864,1
2022-10-03,19044.068359,19653.542969,19025.226562,19623.580078,30484729489,10/03/2022,-0.003864,1
2022-10-04,19623.583984,20380.343750,19523.839844,20336.843750,35887278685,10/04/2022,-0.003864,1


In [45]:
mydf2.dtypes

open                   float64
high                   float64
low                    float64
close                  float64
volume                   int64
Date_mesure             object
sentiment_score_avg    float64
sentiment_score_maj      int64
dtype: object

In [46]:
mydf2.drop("Date_mesure", axis=1, inplace=True)

In [47]:
mydf2

,open,high,low,close,volume,sentiment_score_avg,sentiment_score_maj
date,,,,,,,
2022-10-01,19431.105469,19471.154297,19231.082031,19312.095703,18719537670,-0.003864,1
2022-10-02,19311.849609,19370.308594,18970.621094,19044.107422,20765955327,-0.003864,1
2022-10-03,19044.068359,19653.542969,19025.226562,19623.580078,30484729489,-0.003864,1
2022-10-04,19623.583984,20380.343750,19523.839844,20336.843750,35887278685,-0.003864,1


In [48]:
mydf2.dtypes

open                   float64
high                   float64
low                    float64
close                  float64
volume                   int64
sentiment_score_avg    float64
sentiment_score_maj      int64
dtype: object

In [50]:
mydf2.corr()

,open,high,low,close,volume,sentiment_score_avg,sentiment_score_maj
open,1.000000,0.584828,0.841742,0.474156,0.158805,NaN,NaN
high,0.584828,1.000000,0.868327,0.983296,0.889269,NaN,NaN
low,0.841742,0.868327,1.000000,0.839376,0.560022,NaN,NaN
close,0.474156,0.983296,0.839376,1.000000,0.915977,NaN,NaN
volume,0.158805,0.889269,0.560022,0.915977,1.000000,NaN,NaN
sentiment_score_avg,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sentiment_score_maj,NaN,NaN,NaN,NaN,NaN,NaN,NaN
